<a href="https://colab.research.google.com/github/kmalicekim/NLP_practice_mine/blob/master/5.BOW_%EA%B8%B0%EB%B0%98%EC%9D%98_%EB%AC%B8%EC%84%9C%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## BOW 기반의 문서 분류

## 1. 20뉴스그룹 데이터 준비 및 특성 추출

In [1]:
from sklearn.datasets import fetch_20newsgroups

# 20개의 토픽 중 선택하고자 하는 토픽을 리스트로 생성
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

# 학습 데이터셋 가져옴
newsgroups_train = fetch_20newsgroups(subset = 'train', 
                                      remove = ('headers', 'footers', 'quotes'), #메일 내용에서 hint가 되는 부분 삭제 (순수하게 내용만으로 분류하기위해)
                                      categories = categories)

# 검증 데이터셋 가져옴
newsgroups_test = fetch_20newsgroups(subset = 'test', 
                                     remove = ('headers', 'footers', 'quotes'),
                                     categories = categories)

print('#Train set size:', len(newsgroups_train.data))
print('#Test set size:', len(newsgroups_test.data))
print('#Selected categories:', newsgroups_train.target_names)
print('#Train labels:', set(newsgroups_train.target))


#Train set size: 2034
#Test set size: 1353
#Selected categories: ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']
#Train labels: {0, 1, 2, 3}


In [2]:
print('#Train set text samples:', newsgroups_train.data[0])
print('#Train set label samples:', newsgroups_train.target[0])
print('#Test set text samples:', newsgroups_test.data[0])
print('#Test set label samples:', newsgroups_test.target[0])

#Train set text samples: Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych
#Train set label samples: 1
#Test set text samples: TRry the SKywatch project in  Arizona.
#Test set label samples: 2


In [3]:
X_train = newsgroups_train.data #학습 데이터셋 문서
y_train = newsgroups_train.target #학습 데이터셋 라벨

X_test = newsgroups_test.data #검증 데이터셋 문서
y_test = newsgroups_test.target #검증 데이터셋 라벨


from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=2000, min_df=5, max_df=0.5) #5이상 & 전체의 50% 안넘어가는 것만

X_train_cv = cv.fit_transform(X_train) #train set을 변환
print('Train set dimension:', X_train_cv.shape)

X_test_cv = cv.fit_transform(X_test) #test set을 변환
print('Test set dimension:', X_test_cv.shape)

Train set dimension: (2034, 2000)
Test set dimension: (1353, 2000)


In [4]:
print(len(cv.get_feature_names()))

for word, count in zip(cv.get_feature_names_out()[:100], X_train_cv[0].toarray()[0,:100]):
  print(word, ':', count, end=', ')

2000
00 : 0, 000 : 0, 10 : 0, 100 : 0, 11 : 0, 12 : 0, 128 : 0, 129 : 0, 13 : 0, 130 : 0, 14 : 0, 15 : 0, 150 : 0, 16 : 0, 17 : 0, 18 : 0, 19 : 0, 192 : 0, 1987 : 0, 1988 : 0, 1989 : 0, 1990 : 0, 1991 : 0, 1992 : 0, 1993 : 0, 20 : 0, 200 : 0, 21 : 0, 22 : 0, 23 : 0, 24 : 0, 24bit : 0, 25 : 0, 256 : 0, 26 : 0, 27 : 0, 275 : 0, 28 : 0, 286 : 0, 29 : 0, 2d : 0, 30 : 0, 300 : 0, 31 : 0, 32 : 0, 33 : 0, 34 : 0, 35 : 0, 36 : 0, 386 : 0, 3d : 0, 40 : 0, 41 : 0, 42 : 0, 44 : 0, 45 : 0, 48 : 0, 50 : 0, 500 : 0, 60 : 0, 61 : 0, 64 : 0, 65 : 0, 70 : 0, 75 : 0, 80 : 0, 800 : 0, 90 : 0, 91 : 0, 92 : 0, 93 : 1, 95 : 0, ability : 1, able : 0, abortion : 0, about : 0, above : 0, absolute : 0, absolutely : 0, ac : 0, accept : 0, accepted : 0, access : 0, according : 0, account : 0, across : 0, act : 0, action : 0, actions : 0, active : 0, activity : 0, actual : 0, actually : 0, ad : 0, add : 0, added : 0, addition : 0, additional : 0, address : 0, addresses : 0, 

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## 2. 머신러닝과 문서 분류 과정에 대한 이해

## 3. 나이브 베이즈 분류기 (Naive Bayse Classifier)를 이용한 문서 분류

In [5]:
from sklearn.naive_bayes import MultinomialNB #sklearn이 제공하는 MultinomialNB 사용
NB_clf = MultinomialNB() # 분류기 선언

NB_clf.fit(X_train_cv, y_train) #train set을 이용하여 분류기(classifier)를 학습

print('Train set score: {:.3f}'.format(NB_clf.score(X_train_cv, y_train))) #train set에 대한 예측정확도 확인
print('Test set score: {:.3f}'.format(NB_clf.score(X_test_cv, y_test))) #test set에 대한 예측정확도 확인

Train set score: 0.824
Test set score: 0.368


In [6]:
print('#First document and label in test data:', X_test[0], y_test[0])
print('#Second document and label in test data:', X_test[1], y_test[1])

pred = NB_clf.predict(X_test_cv[:2]) 

print('#Predicted labels:', pred)
print('#Predicted categories:', newsgroups_train.target_names[pred[0]], newsgroups_train.target_names[pred[1]])

#First document and label in test data: TRry the SKywatch project in  Arizona. 2
#Second document and label in test data: The Vatican library recently made a tour of the US.
 Can anyone help me in finding a FTP site where this collection is 
 available. 1
#Predicted labels: [1 2]
#Predicted categories: comp.graphics sci.space


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

#CountVectorizer와 동일한 인수 사용
tfidf = TfidfVectorizer(max_features=2000, min_df=5, max_df=0.5)
X_train_tfidf = tfidf.fit_transform(X_train) #train set을 변환
X_test_tfidf = tfidf.transform(X_test) #test set을 변환

NB_clf.fit(X_train_tfidf, y_train)
print('Train set score: {:.3f}'.format(NB_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(NB_clf.score(X_test_tfidf, y_test)))

Train set score: 0.862
Test set score: 0.741


In [8]:
import numpy as np

def top10_features(classifier, vectorizer, categories):
  feature_names = np.asarray(vectorizer.get_feature_names_out())  # asarray() : convert an given input to an array
  for i, category in enumerate(categories):
    # 역순으로 정렬하기 위해 계수에 음수를 취해서 정렬 후 앞에서부터 10개의 값을 반환
    top10 = np.argsort(-classifier.coef_[i])[:10]

    # 카테고리와 영향이 큰 특성 10개 출력
    print("{}:{}".format(category, ", ".join(feature_names[top10])))

top10_features(NB_clf, tfidf, newsgroups_train.target_names)    

alt.atheism:you, not, are, be, this, have, as, what, they, if
comp.graphics:you, on, graphics, this, have, any, can, or, with, thanks
sci.space:space, on, you, be, was, this, as, they, have, are
talk.religion.misc:you, not, he, are, as, this, be, god, was, they


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `coef_` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)


## 4. 로지스틱 회귀분석을 이용한 문서 분류

In [9]:
from sklearn.linear_model import LogisticRegression 

# count vector에 대해 regression을 해서 NB와 비교
LR_clf = LogisticRegression() # 분류기 선언
LR_clf.fit(X_train_tfidf, y_train)
print('Train set score: {:.3f}'.format(LR_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(LR_clf.score(X_test_tfidf, y_test)))

Train set score: 0.930
Test set score: 0.734


## 릿지 회귀 (ridge regression)

In [10]:
from sklearn.linear_model import RidgeClassifier

ridge_clf = RidgeClassifier()
ridge_clf.fit(X_train_tfidf, y_train)
print('Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))

Train set score: 0.960
Test set score: 0.735


In [11]:
import numpy as np
from sklearn.model_selection import train_test_split 

X_train_ridge, X_val_ridge, y_train_ridge, y_val_ridge = train_test_split(
    X_train_tfidf, y_train, test_size=0.2, random_state=42)

max_score = 0
max_alpha = 0

# ridge에서 가중치(w)의 절댓값을 가능한 한 작게 만드려고 함. 과대적합이 되지 않도록 모델을 강제로 제한(L2규제)
# alpha 매개변수로 훈련 세트의 성능 대비 모델을 얼마나 단순화할지를 지정 가능 (기본값 alpha = 1.0)
# alpha 값을 높이면 계수를 0에 더 가깝게 만들어 훈련세트의 성능은 나빠지지만 일반화에는 쉬워짐

for alpha in np.arange(0.1, 10, 0.1): #alpha를 0.1부터 10까지 0.1씩 증가 
  ridge_clf = RidgeClassifier(alpha=alpha)
  ridge_clf.fit(X_train_ridge, y_train_ridge) 
  score = ridge_clf.score(X_val_ridge, y_val_ridge)
  if score > max_score:
    max_score = score
    max_alpha = alpha

print('Max alpha {:.3f} at max validation score {:.3f}'.format(max_alpha, max_score))    


Max alpha 1.600 at max validation score 0.826


In [12]:
ridge_clf = RidgeClassifier(alpha=1.6)
ridge_clf.fit(X_train_tfidf, y_train)

print('Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))

Train set score: 0.948
Test set score: 0.739


In [13]:
top10_features(ridge_clf, tfidf, newsgroups_train.target_names) #ridge 분류기가 훨씬 좋은 결과물 보여줌

alt.atheism:bobby, religion, atheism, atheists, motto, punishment, islam, deletion, islamic, satan
comp.graphics:graphics, computer, 3d, file, image, hi, 42, using, screen, looking
sci.space:space, orbit, nasa, spacecraft, moon, sci, launch, flight, funding, idea
talk.religion.misc:christian, christians, fbi, blood, order, jesus, objective, children, christ, hudson


---

## 라쏘 회귀분석(Lasso regression)을 이용한 특성 선택 (feature selection)

In [14]:
lasso_clf = LogisticRegression(penalty='l1', solver='liblinear', C=1) #Lasso는 동일한 LogisticRegression을 사용하면서 매개변수로 지정. penalty='l1'으로 해주어야 함
lasso_clf.fit(X_train_tfidf, y_train)

print('#Train set score: {:.3f}'.format(lasso_clf.score(X_train_tfidf, y_train)))
print('#Test set score: {:.3f}'.format(lasso_clf.score(X_test_tfidf, y_test)))


# 계수(coefficient)중에서 0이 아닌 것들의 개수 출력
print('#Used features count: {}'.format(np.sum(lasso_clf.coef_ !=0)), 'out of', X_train_tfidf.shape[1])


#Train set score: 0.819
#Test set score: 0.724
#Used features count: 437 out of 2000


In [15]:
top10_features(lasso_clf, tfidf, newsgroups_train.target_names)

alt.atheism:bobby, atheism, atheists, islam, religion, islamic, motto, atheist, satan, vice
comp.graphics:graphics, image, 3d, file, computer, hi, video, files, looking, sphere
sci.space:space, orbit, launch, nasa, spacecraft, flight, moon, dc, shuttle, solar
talk.religion.misc:fbi, christian, christians, christ, order, jesus, children, objective, context, blood


---

## 5. 결정트리 등을 이용한 기타 문서 분류 방법

In [16]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

tree = DecisionTreeClassifier(random_state = 7)
tree.fit(X_train_tfidf, y_train)
print('#Decision Tree train set score: {:.3f}'.format(tree.score(X_train_tfidf, y_train)))
print('#Decision Tree test set score: {:.3f}'.format(tree.score(X_test_tfidf, y_test)))

forest = RandomForestClassifier(random_state = 7)
forest.fit(X_train_tfidf, y_train)
print('#Decision Tree train set score: {:.3f}'.format(forest.score(X_train_tfidf, y_train)))
print('#Decision Tree test set score: {:.3f}'.format(forest.score(X_test_tfidf, y_test)))

gb = GradientBoostingClassifier(random_state = 7)
gb.fit(X_train_tfidf, y_train)
print('#Decision Tree train set score: {:.3f}'.format(gb.score(X_train_tfidf, y_train)))
print('#Decision Tree test set score: {:.3f}'.format(gb.score(X_test_tfidf, y_test)))


#Decision Tree train set score: 0.977
#Decision Tree test set score: 0.536
#Decision Tree train set score: 0.977
#Decision Tree test set score: 0.685
#Decision Tree train set score: 0.933
#Decision Tree test set score: 0.696


In [17]:
# sort & lambda 예제
data_list = ['but','i','wont','hesitate','no','more','no','more','it','cannot','wait','im','yours']

data_list = list(set(data_list))

data_list.sort()
print(data_list) 

data_list.sort(key=lambda x:len(x))
print(data_list)


['but', 'cannot', 'hesitate', 'i', 'im', 'it', 'more', 'no', 'wait', 'wont', 'yours']
['i', 'im', 'it', 'no', 'but', 'more', 'wait', 'wont', 'yours', 'cannot', 'hesitate']


In [18]:
sorted_feature_importances = sorted(zip(tfidf.get_feature_names_out(), gb.feature_importances_), 
                                    key=lambda x:x[1], reverse=True)

for feature, value in sorted_feature_importances[:40]:
  print('{}:{}'.format(feature, value), end=', ')


space:0.12606815556503398, graphics:0.07955738535484687, atheism:0.023543258791466362, thanks:0.023046507462732756, file:0.02055581240238831, orbit:0.020063273938036058, jesus:0.01814149212443511, god:0.017709491540890448, hi:0.016857094927865728, nasa:0.015308260768876465, image:0.015023452519852164, files:0.013797145686816017, christ:0.010426583770790022, moon:0.010376173054679735, bobby:0.010225361352506715, launch:0.010061136447910934, looking:0.009733618292917431, christian:0.009663039306452306, atheists:0.00940203835982551, christians:0.008981046753380314, fbi:0.008827368636033589, 3d:0.007819020701620837, you:0.007751791923437367, not:0.007535276098728061, islamic:0.007255118998822582, religion:0.007182617149762632, spacecraft:0.007074189152295397, flight:0.006899060277543797, computer:0.006698870635101267, islam:0.006529364772733121, ftp:0.0062457492372051055, color:0.005513872126424359, software:0.004918770389689076, atheist:0.004805134811116732, card:0.004767940213268275, peo

---

## 6. 성능을 높이기 위한 방법들

In [19]:
# 필요한 library들을 import
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
cachedStopWords = stopwords.words("english")

from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
import re

RegTok = RegexpTokenizer("[\w']{3,}") #정규표현식으로 토크나이저 정의
english_stops = set(stopwords.words('english')) #영어 불용어를 가져옴

def tokenizer(text):
  tokens = RegTok.tokenize(text.lower())
  words = [word for word in tokens if (word not in english_stops) and len(word) > 2 ]
  features = (list(map(lambda token : PorterStemmer().stem(token), words)))
  return features

tfidf = TfidfVectorizer(tokenizer=tokenizer, max_features=2000, min_df=5, max_df=0.5)
X_train_tfidf = tfidf.fit_transform(X_train) #train set을 변환
X_test_tfidf = tfidf.transform(X_test) # test set을 변환 --- test set은 fit_transform 이 아니라 transform 만!

#tfidf vector를 이용해서 분류기 학습
LR_clf = LogisticRegression()
LR_clf.fit(X_train_tfidf, y_train)

print('#Train set score: {:.3f}'.format(LR_clf.score(X_train_tfidf, y_train)))
print('#Test set score: {:.3f}'.format(LR_clf.score(X_test_tfidf, y_test)))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


#Train set score: 0.930
#Test set score: 0.751


In [20]:
len(LR_clf.coef_[0])

2000

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(tokenizer=tokenizer)

X_train_tfidf = tfidf.fit_transform(X_train)
print('#Train set dimension:', X_train_tfidf.shape)
X_test_tfidf = tfidf.transform(X_test)
print('#Test set dimension:', X_test_tfidf.shape)

ridge_clf = RidgeClassifier(alpha=2.4)
ridge_clf.fit(X_train_tfidf, y_train)
print('#Train set score:{:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('#Test set score:{:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))

NB_clf = MultinomialNB(alpha=0.01)
NB_clf.fit(X_train_tfidf, y_train)
print('#Train set score: {:.3f}'.format(NB_clf.score(X_train_tfidf, y_train)))
print('#Test set score: {:.3f}'.format(NB_clf.score(X_test_tfidf, y_test)))

#Train set dimension: (2034, 20085)
#Test set dimension: (1353, 20085)
#Train set score:0.968
#Test set score:0.768
#Train set score: 0.971
#Test set score: 0.793


---

## 7. 카운트 기반의 문제점과 N-gram

In [22]:
cachedStopWords = stopwords.words("english")
tfidf = TfidfVectorizer(token_pattern="[a-zA-Z]{3,}", #토큰화를 위한 정규식
                        decode_error='ignore',
                        lowercase=True,
                        stop_words=stopwords.words('english'),
                        max_df=0.5, min_df=2)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

print(X_train_tfidf.shape)

(2034, 11346)


In [23]:
from sklearn.linear_model import RidgeClassifier

ridge_clf = RidgeClassifier()
ridge_clf.fit(X_train_tfidf, y_train) #학습
print('Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))


Train set score: 0.976
Test set score: 0.769


In [24]:
# bigram으로 설정하기

tfidf = TfidfVectorizer(token_pattern="[a-zA-Z]{3,}", 
                        decode_error='ignore',
                        lowercase=True,
                        stop_words=stopwords.words('english'),
                        ngram_range=(1,2), #바이그램 설정
                        max_df=0.5, min_df=2)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

print(X_train_tfidf.shape)

(2034, 26348)


In [25]:
bigram_features = [f for f in tfidf.get_feature_names_out() if len(f.split()) > 1]

print('bi-gram samples:', bigram_features[:10])

ridge_clf.fit(X_train_tfidf, y_train) #학습
print('Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))

bi-gram samples: ['aas american', 'ability means', 'ability pass', 'able accept', 'able afford', 'able control', 'able convince', 'able draw', 'able establish', 'able find']
Train set score: 0.976
Test set score: 0.776


In [26]:
# trigram으로 설정하기

tfidf = TfidfVectorizer(token_pattern= "[a-zA-Z']{3,}", 
                        decode_error ='ignore', 
                        lowercase=True, 
                        stop_words = stopwords.words('english'),
                        ngram_range=(1, 3),
                        max_df=0.5,
                        min_df=2)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)
print(X_train_tfidf.shape)

trigram_features = [f for f in tfidf.get_feature_names_out() if len(f.split()) > 2]
print('tri-gram samples:', trigram_features[:10])

ridge_clf.fit(X_train_tfidf, y_train) #학습
print('Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))


(2034, 32943)
tri-gram samples: ["'em better shots", "'expected errors' basically", "'karla' next one", "'nodis' password also", "'official doctrine think", "'ok see warning", "'what's moonbase good", 'aas american astronautical', 'ability means infallible', 'able accept donations']
Train set score: 0.976
Test set score: 0.775


---

## 8. 한글 문서의 분류

In [27]:
import pandas as pd
df = pd.read_csv('/daum_movie_review.csv')
df.head(5)

,review,rating,date,title
0,돈 들인건 티가 나지만 보는 내내 하품만,1,2018.10.29,인피니티 워
1,몰입할수밖에 없다. 어렵게 생각할 필요없다. 내가 전투에 참여한듯 손에 땀이남.,10,2018.10.26,인피니티 워
2,이전 작품에 비해 더 화려하고 스케일도 커졌지만.... 전국 맛집의 음식들을 한데 ...,8,2018.10.24,인피니티 워
3,이 정도면 볼만하다고 할 수 있음!,8,2018.10.22,인피니티 워
4,재미있다,10,2018.10.20,인피니티 워


In [28]:
df.title.value_counts()

신과함께      4947
택시운전사     2322
인피니티 워    2042
범죄도시      1939
곤지암       1547
라라랜드      1150
코코         778
Name: title, dtype: int64

In [32]:
from sklearn.model_selection import train_test_split

# split data and labels into a training and a test set
X_train, X_test, y_train, y_test = train_test_split(df.review, df.title, random_state=0)

# 비율을 지정하지 않으면 75:25로 분할됨
print('#Train set size:', len(X_train)) #실제로 몇개의 특성이 사용되었는지 확인
print('#Test set size:', len(X_test))

#Train set size: 11043
#Test set size: 3682


In [35]:
!pip install konlpy
from konlpy.tag import Okt #konlpy에서 Twitter 형태소 분석기 import

okt = Okt()
print(okt.morphs(X_train[1])) #두번째 리뷰에 대해 형태소 단위로 tokenize
print(okt.nouns(X_train[1]))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
['몰입', '할수밖에', '없다', '.', '어렵게', '생각', '할', '필요없다', '.', '내', '가', '전투', '에', '참여', '한', '듯', '손', '에', '땀', '이남', '.']
['몰입', '생각', '내', '전투', '참여', '듯', '손', '땀', '이남']


In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Twitter 형태소분석기에서 명사만 추출하는 함수를 tokenizer로 이용
tfidf = TfidfVectorizer(tokenizer=okt.nouns, max_features=2000, min_df=5, max_df=0.5)

X_train_tfidf = tfidf.fit_transform(X_train) # train data 변환 -> tfidf vector
X_test_tfidf = tfidf.transform(X_test) # test data 변환 -> tfidf vector

# logistic regression 분류기 선언
# 충분한 학습을 위해 max_iter(경사하강법 반복횟수) 를 1000으로 설정. 기본은 100
clf = LogisticRegression(max_iter=1000) 
clf.fit(X_train_tfidf, y_train) # 분류기 학습

print('#Train set score: {:.3f}'.format(clf.score(X_train_tfidf, y_train)))
print('#Test set score: {:.3f}'.format(clf.score(X_test_tfidf, y_test)))

#Train set score: 0.756
#Test set score: 0.694


In [38]:
print('실제영화제목, 예측한 제목, 리뷰')
for content in zip(y_test[:10], clf.predict(X_test_tfidf[:10]), X_test[:10]):
  print(content)

실제영화제목, 예측한 제목, 리뷰
('범죄도시', '신과함께', '오랜만에 잼나는 영화 봤습니다.  다음에 더 재미있는 영화 기대하겠습니다.')
('범죄도시', '범죄도시', '조연들이 눈에 박힌다. 간만에 집중 ㅎ')
('코코', '코코', '대감동을 선사. 인사이드 아웃을 잇는 픽사의 감동스토리. 신과함께의 멕시코판이라고나할까요??')
('신과함께', '신과함께', '돈이 안아까웠던 영화ᆞᆞ  정말 좋았다')
('신과함께', '신과함께', '역시 김용화감독이 영화는 잘 만들어요. 이제 VFX 제작 부문도 헐리우드 수준 이상입니다.')
('택시운전사', '택시운전사', '민주화를 위해 힘써주신 분들께 감사하는 마음으로 살아야겠다.')
('신과함께', '신과함께', '잠만 자다 왔음')
('신과함께', '신과함께', '오랜만에 잼있고 좋은 영화를 봤다')
('범죄도시', '신과함께', '잼남')
('범죄도시', '인피니티 워', '대박~~')


---

## 성능을 개선하기 위한 노력

In [40]:
tfidf = TfidfVectorizer(tokenizer=okt.morphs, max_features=2000, min_df=5, max_df=0.5) # 명사 대신 모든 형태소 사용
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_tfidf, y_train)
print('#Train set score: {:.3f}'.format(clf.score(X_train_tfidf, y_train)))
print('#Test set score: {:.3f}'.format(clf.score(X_test_tfidf, y_test)))

#Train set score: 0.777
#Test set score: 0.695


In [44]:
def twit_tokenizer(text): #전체를 다 사용 X. 명사, 동사, 형용사만 사용
  target_tags = ['Noun', 'Verb', 'Adjective']
  result = []

  for word, tag in okt.pos(text, norm=True, stem=True):
    if tag in target_tags:
      result.append(word)
  return result


tfidf = TfidfVectorizer(tokenizer=twit_tokenizer, max_features=2000, min_df=5, max_df=0.5)   
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_tfidf, y_train)
print('#Train set score: {:.3f}'.format(clf.score(X_train_tfidf, y_train)))
print('#Test set score: {:.3f}'.format(clf.score(X_test_tfidf, y_test)))

#Train set score: 0.784
#Test set score: 0.712


In [45]:
# 모든 형태소를 다 사용하고 품사를 알 수 있도록 한다면?

def twit_tokenizer2(text):
  result = []

  for word, tag in okt.pos(text, norm=True, stem=True):
    result.append('/'.join([word, tag])) #단어의 품사를 구분할 수 있도록 함
  return result


print(twit_tokenizer2(X_train[1]))

['몰입/Noun', '하다/Verb', '없다/Adjective', './Punctuation', '어렵다/Adjective', '생각/Noun', '하다/Verb', '필요없다/Adjective', './Punctuation', '내/Noun', '가/Josa', '전투/Noun', '에/Josa', '참여/Noun', '한/Determiner', '듯/Noun', '손/Noun', '에/Josa', '땀/Noun', '이남/Noun', './Punctuation']


In [46]:
tfidf = TfidfVectorizer(tokenizer=twit_tokenizer2, max_features=2000, min_df=5, max_df=0.5) 
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test) 

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_tfidf, y_train)

print('#Train set score: {:.3f}'.format(clf.score(X_train_tfidf, y_train)))
print('#Test set score: {:.3f}'.format(clf.score(X_test_tfidf, y_test)))

#Train set score: 0.789
#Test set score: 0.718


In [47]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train_ridge, X_val_ridge, y_train_ridge, y_val_ridge = train_test_split(
    X_train_tfidf, y_train, test_size = 0.2, random_state=42)


max_score = 0
max_alpha = 5

for alpha in np.arange(0.1, 10, 0.1):        # alpha를 0.1부터 10까지 0.1씩 증가
  ridge_clf = RidgeClassifier(alpha=alpha)   # 릿지분류기 선언
  ridge_clf.fit(X_train_ridge, y_train_ridge) # 학습
  score = ridge_clf.score(X_val_ridge, y_val_ridge) # 검정 데이터셋에 대해 정확도 측정

  if score > max_score:
    max_score = score
    max_alpha = alpha

print('#Max alpha {:.3f} at max validation score {:.3f}'.format(max_alpha, max_score))


#Max alpha 1.600 at max validation score 0.727


In [48]:
from sklearn.linear_model import RidgeClassifier

ridge_clf = RidgeClassifier(alpha=1.6) 
ridge_clf.fit(X_train_tfidf, y_train)

print('#Ridge Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('#Ridge Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))

from sklearn.linear_model import LogisticRegression
import numpy as np

lasso_clf = LogisticRegression(penalty='l1', solver='liblinear', C=0.5)
lasso_clf.fit(X_train_tfidf, y_train)

print('#Lasso Train set score: {:.3f}'.format(lasso_clf.score(X_train_tfidf, y_train)))
print('#Lasso Test set score: {:.3f}'.format(lasso_clf.score(X_test_tfidf, y_test)))
print('#Used features count: {}'.format(np.sum(lasso_clf.coef_ !=0)), 'out of', X_train_tfidf.shape[1])


#Ridge Train set score: 0.807
#Ridge Test set score: 0.726
#Lasso Train set score: 0.703
#Lasso Test set score: 0.696
#Used features count: 957 out of 2000


In [54]:
print(X_train_tfidf.shape, y_train.shape)
print(X_test_tfidf.shape, y_test.shape)
np.sum(df.title.value_counts())

(11043, 2000) (11043,)
(3682, 2000) (3682,)


14725

In [57]:
from sklearn.naive_bayes import MultinomialNB

NB_clf = MultinomialNB(alpha=0.1)
NB_clf.fit(X_train_tfidf, y_train)

print('Train set score: {:.3f}'.format(NB_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(NB_clf.score(X_test_tfidf, y_test)))

Train set score: 0.768
Test set score: 0.704
